In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, DoubleType, DataType, BooleanType, TimestampType
from pyspark.sql import SparkSession

In [0]:
spark.conf.set("fs.azure.account.auth.type.databriconnect.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.databriconnect.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.databriconnect.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.databriconnect.dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.databriconnect.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
display(dbutils.fs.ls("abfss://silver@databriconnect.dfs.core.windows.net/SalesLT"))

path,name,size,modificationTime
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/Address/,Address/,0,1746464908000
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/Customer/,Customer/,0,1746464918000
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/CustomerAddress/,CustomerAddress/,0,1746464921000
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/Product/,Product/,0,1746464924000
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/ProductCategory/,ProductCategory/,0,1746464926000
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/ProductDescription/,ProductDescription/,0,1746464928000
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/ProductModel/,ProductModel/,0,1746464930000
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/ProductModelProductDescription/,ProductModelProductDescription/,0,1746464932000
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/SalesOrderDetail/,SalesOrderDetail/,0,1746464934000
abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/SalesOrderHeader/,SalesOrderHeader/,0,1746464936000


In [0]:
table_name = []

for i in dbutils.fs.ls("abfss://silver@databriconnect.dfs.core.windows.net/SalesLT"):
    table_name.append(i.name[:-1])

print(table_name)

['Address', 'Customer', 'CustomerAddress', 'Product', 'ProductCategory', 'ProductDescription', 'ProductModel', 'ProductModelProductDescription', 'SalesOrderDetail', 'SalesOrderHeader']


In [0]:
for name in table_name:
    path = "abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/" + name 
    df = spark.read.format('delta').load(path)

    column_name = df.columns
    for old_col_name in column_name:
        new_col_name = "".join(['_' + char if char.isupper() and not old_col_name[i - 1].isupper() else char for i, char in enumerate(old_col_name)]).lstrip("_")

        df = df.withColumnRenamed(old_col_name,new_col_name)

    output_path = 'abfss://gold@databriconnect.dfs.core.windows.net/SalesLT/' + name + '/'
    df.write.format("delta").mode("overwrite").save(output_path)